In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator



c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Directories
train_dir = "DataSet_V3_7525/train"
val_dir = "DataSet_V3_7525/val"
test_dir = "DataSet_V3_7525/test"

# Rescale images
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Train data
train_data = train_datagen.flow_from_directory(
    directory=train_dir,
    batch_size=32,
    target_size=(299, 299),
    class_mode="categorical",
    shuffle=True  # Ensure shuffling
)

# Validation data
val_data = val_datagen.flow_from_directory(
    directory=val_dir,
    batch_size=32,
    target_size=(299, 299),
    class_mode="categorical"
)

# Test data
test_data = test_datagen.flow_from_directory(
    directory=test_dir,
    batch_size=32,
    target_size=(299, 299),
    class_mode="categorical",
    shuffle=False  # No need to shuffle test data
)

print(f"Train dataset size: {train_data.samples}")
print(f"Validation dataset size: {val_data.samples}")
print(f"Test dataset size: {test_data.samples}")

Found 3897 images belonging to 14 classes.
Found 1305 images belonging to 14 classes.
Found 1740 images belonging to 14 classes.
Train dataset size: 3897
Validation dataset size: 1305
Test dataset size: 1740


In [5]:
steps_per_epoch = train_data.samples // train_data.batch_size
validation_steps = val_data.samples // val_data.batch_size
print(steps_per_epoch);

121


In [4]:
# Steps per epoch
steps_per_epoch = train_data.samples // train_data.batch_size
validation_steps = val_data.samples // val_data.batch_size

# Number of classes
num_classes = train_data.num_classes
print(f"Number of classes: {num_classes}")

# 1. Create a base model with tf.keras.applications
base_model = tf.keras.applications.InceptionV3(include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False  # Freeze base model

# 2. Build custom model
inputs = tf.keras.layers.Input(shape=(299, 299, 3), name="input-layer")
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="output-layer")(x)
model = tf.keras.Model(inputs, outputs)

# 3. Compile model
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Smaller learning rate
    metrics=["accuracy"]
)

# 4. Train the model
history = model.fit(
    train_data,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_data,
    validation_steps=validation_steps
)

Number of classes: 14


C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 1121s 9s/step - accuracy: 0.2058 - loss: 2.5149 - val_accuracy: 0.5773 - val_loss: 1.8203
Epoch 2/10
  1/121 ━━━━━━━━━━━━━━━━━━━━ 4:09 2s/step - accuracy: 0.7500 - loss: 1.7048

c:\ProgramData\Anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


121/121 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.7500 - loss: 1.7048 - val_accuracy: 0.4400 - val_loss: 1.8525
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 1085s 9s/step - accuracy: 0.6395 - loss: 1.6608 - val_accuracy: 0.7484 - val_loss: 1.3097
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.7500 - loss: 1.3539 - val_accuracy: 0.6800 - val_loss: 1.5010
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 1086s 9s/step - accuracy: 0.7736 - loss: 1.2116 - val_accuracy: 0.8109 - val_loss: 1.0219
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8750 - loss: 0.9423 - val_accuracy: 0.8400 - val_loss: 1.0373
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 946s 8s/step - accuracy: 0.8173 - loss: 0.9682 - val_accuracy: 0.8422 - val_loss: 0.8454
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.8125 - loss: 0.8526 - val_accuracy: 0.8000 - val_loss: 0.8907
Epoch 9/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 1006s 8s/step - accuracy: 0.8508 - loss: 0.7917 - val_accuracy: 0.

In [6]:
# 5. Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test accuracy: {test_accuracy:.2f}")

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


55/55 ━━━━━━━━━━━━━━━━━━━━ 464s 8s/step - accuracy: 0.8503 - loss: 0.6634
Test accuracy: 0.86


In [7]:
# 6. Save the model
model.save("BirdSpecies_V3_7525.h5")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the model
model = load_model('BirdSpecies_V3_7525.h5')

# Path to test data
test_dir = r'E:\CAPSTONE\Bird_Species_Classification\DataSet_V3_7525\test'

# Image data generator for preprocessing
test_datagen = ImageDataGenerator()

# Load test data
test_data = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(299, 299),  # Adjust size as per your model's input
    batch_size=32,
    class_mode='categorical',  # Use 'binary' if you have two classes
    shuffle=False  # Do not shuffle to match predictions with true labels
)

# Get class indices
class_indices = test_data.class_indices
class_labels = list(class_indices.keys())

# Predict on test data
predictions = model.predict(test_data)
y_pred = np.argmax(predictions, axis=1)
y_true = test_data.classes

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Classification report
report = classification_report(y_true, y_pred, target_names=class_labels)
print("Classification Report:\n", report)


: 